In [1]:
import os 


In [2]:
%cd /home/lab/Workspace/e2e-mlflow-dvc

/home/lab/Workspace/e2e-mlflow-dvc


In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create artifacts root directory
        create_directories([self.config["artifacts_root"]])  # Access dictionary key correctly

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        # Access prepare_base_model section of config
        config = self.config["prepare_base_model"]  

        # Create the directory for the base model
        create_directories([config["root_dir"]])

        # Build PrepareBaseModelConfig object
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config["root_dir"]),
            base_model_path=Path(config["base_model_path"]),
            updated_base_model_path=Path(config["updated_base_model_path"]),
            params_image_size=self.params["IMAGE_SIZE"],
            params_learning_rate=self.params["LEARNING_RATE"],
            params_include_top=self.params["INCLUDE_TOP"],
            params_weights=self.params["WEIGHTS"],
            params_classes=self.params["CLASSES"]
        )

        return prepare_base_model_config

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from pathlib import Path

In [18]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        try:
            # Validate input shape
            input_shape = tuple(self.config.params_image_size)
            
            # Print configuration for debugging
            print("Model Configuration:")
            print(f"Input Shape: {input_shape}")
            print(f"Weights: '{self.config.params_weights}'")  # Debugging print
            print(f"Include Top: {self.config.params_include_top}")
            print(f"Classes: {self.config.params_classes}")

            # Strip any whitespace from weights
            weights = self.config.params_weights.strip() if self.config.params_weights else None
            if weights not in {"imagenet", None}:
                raise ValueError(f"Invalid weights: '{weights}'. Must be 'imagenet' or None.")
            
            self.model = tf.keras.applications.VGG16(
                input_shape=input_shape,
                weights=weights,
                include_top=self.config.params_include_top,
                classes=self.config.params_classes if self.config.params_include_top else None
            )
            
            self.save_model(path=self.config.base_model_path, model=self.model)
        except Exception as e:
            print(f"Error in get_base_model: {e}")
            raise
            
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        model.save(path)

In [17]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-01-23 15:12:51,639: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-23 15:12:51,649: ERROR: common: An error occurred while reading YAML file: while scanning a simple key
  in "params.yaml", line 7, column 1
could not find expected ':'
  in "params.yaml", line 8, column 1]


ScannerError: while scanning a simple key
  in "params.yaml", line 7, column 1
could not find expected ':'
  in "params.yaml", line 8, column 1